# Implementation of Algorithm 1 (Long Only) using Interactive Brokers 

### Introduction 

The following code uses Algo 1, which has been rigorously backtested (see Backtester Notebook). The code is to be run once a week. Essentialy, the code uses Algo 1 to rank the stocks of the SP100 and selects the top 10. It then evaluates the current positions of my portfolio, selling the stocks that are no longer in the top 10, and purchasing the new additions, as well as rebalancing the portfolio to make sure it is equally weighted. 

Notes:
1. While I initially used an easier method of buying/selling (market order), I have now created buy/sell functions that use limit orders, starting at the ask/bid price respectively and subsequently increasing/decreasing the price until my order is filled.


2. Looking ahead, I want to automate the entire process so that the user does not need to manually run this script every week.


3. I also intend on building in more functions to track historical progress and see more performance metrics (especially on the efficiency of my buying/selling and the effect of transaction costs).


4. In a few weeks I hope to transition to live trading!

# Basic Imports

In [1]:
import numpy as np
import pandas as pd
import talib
import datetime
import yfinance as yf

### Get list of S&P 100

In [2]:
wiki = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')
SP100 = wiki[2]['Symbol'].values.tolist()

#Temporary Fix to tweak Berkshire Hathaway Name and remove Dow Inc.
SP100[18] = 'BRK-B'

# Variables

In [3]:
STOCKS = SP100[:]
BENCHMARK = '^OEX'

UNIVERSE_SIZE = len(STOCKS)

#Technicals Creator Variables
EMAduration = 10
SMAduration = 100
RSIduration = 14
MACDfast= 12
MACDslow= 26
MACDsignal= 9
MFIduration = 14
RSIcutoff = 20 # distance from 50 (e.g. for cutoff 20, RSI range is 30-70)
MFIcutoff = 30 # distance from 50 (e.g. for cutoff 30, MFI range is 30-70)
basket_size = 10

start_index= max(EMAduration,SMAduration,RSIduration,MACDfast,MACDslow,MACDsignal,MFIduration,RSIcutoff,MFIcutoff)

#Dataset Start and End Dates
end = datetime.datetime.today().date()
start = end - datetime.timedelta(start_index+80)



# Download Data

In [4]:
universe = yf.download(STOCKS, start, end) #,auto_adjust=True)
benchmark = yf.download(BENCHMARK, start, end) # Chicago Options index of SP100

[*********************100%***********************]  101 of 101 completed
[*********************100%***********************]  1 of 1 completed


# Basic Data Cleaning

### Check for NaNs / Correct Them

In [5]:
universe.isna().sum().sum()

0

In [6]:
 universe = universe.fillna(method='bfill')

In [7]:
#Check that the backfill was successful
universe.isna().sum().sum()

0

In [8]:
close = universe['Adj Close']

In [9]:
close.describe()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALL,AMGN,AMT,AMZN,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
count,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,...,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000
mean,333.241353,87.884501,89.985724,195.441045,385.284604,29.033572,95.461850,227.009050,245.052081,2530.697459,...,285.310649,161.884776,108.423078,35.931523,183.910084,55.481825,42.069411,27.276482,122.939435,42.515504
std,67.899340,9.183697,8.777439,25.709568,55.040388,4.965410,7.019816,18.595458,18.285441,474.049642,...,26.352951,18.118714,21.268895,3.343257,15.394186,2.225234,2.517376,3.746787,6.999130,4.110305
min,223.361542,62.785870,62.314651,142.522415,285.000000,18.589884,72.825378,179.810318,177.601761,1676.609985,...,194.024353,113.403679,84.735802,28.257315,135.308548,48.859943,37.489182,22.437664,102.715675,30.254938
25%,281.571243,82.478334,85.751501,175.233204,338.332489,25.254228,92.662500,216.313278,235.195808,2197.482605,...,277.005043,149.389256,93.357382,33.880733,173.369667,54.122215,40.256303,25.002849,118.550125,41.122902
50%,317.996658,90.095367,91.111282,197.371887,384.445007,29.737845,95.764999,230.119194,249.454346,2454.965088,...,290.695007,165.306786,99.773155,36.160000,191.091125,55.773674,41.578566,26.155516,123.173618,42.771214
75%,377.212364,95.451784,95.683537,218.472084,437.062508,31.787501,99.171673,239.658432,258.613297,3006.735107,...,303.593155,174.212498,114.535339,37.437499,195.267300,57.150709,43.118512,28.091166,129.206318,44.392941
max,503.429993,100.830002,102.519997,238.210007,484.429993,42.434875,113.877289,259.213684,271.290009,3346.489990,...,323.700012,194.899994,160.350006,47.040176,208.100006,59.570000,51.624607,41.453659,135.600006,53.680988


# Create Indicators

EMA: Exponential Moving Average

In [10]:
ema = close.apply(lambda c: talib.EMA(c, EMAduration))

SMA: Simple Moving Average

In [11]:
sma = close.apply(lambda c: talib.SMA(c, SMAduration))

RSI: Relative Strength Index

In [12]:
rsi = close.apply(lambda c: talib.RSI(c, RSIduration))

MACD: Moving Average Convergence Divergence

In [13]:
#MACD is the Moving Average Convergence Divergence. MACD Signal is a n day EMA of the MACD.
#The difference between MACD and MACD Signal can be used as signal, therefore, need to track both

macd = pd.DataFrame()
macdsignal = pd.DataFrame()
for stock in close:
    tmacd, tmacdsignal, tmacdhist = talib.MACD(close[stock] , MACDfast, MACDslow, MACDsignal)
    macd[stock] = tmacd
    macdsignal[stock] = tmacdsignal

MFI: Money Flow Index

In [14]:
mfi = pd.DataFrame()
for stock in close:
    tmfi = talib.MFI(universe['High'][stock], universe['Low'][stock], close[stock], universe['Volume'][stock], timeperiod = MFIduration)
    mfi[stock] = tmfi

# Normalize Indicators for Ranking Stock

### EMA Ranking

EMA Ranking Overview:
  
    Buy --> When Close Price is Greater than EMA
    Sell --> When Close Price is Less than EMA
    
    Normalize: The percentage difference between Close Price and EMA

In [15]:
emasignal = (close-ema)/close*100

### SMA Ranking

SMA Ranking Overview:

    Buy --> When EMA is Greater than SMA
    Sell --> When EMA is Less than SMA

    Normalize: The percentage difference between EMA and SMA

In [16]:
smasignal = (ema - sma)/sma*100

### RSI Ranking

RSI Ranking Overview:

    Buy --> When RSI is <30
    Sell --> When RSI is >70
    
    Normalize: RSI should be already normalized because it is a stock's strength RELATIVE to it. For now we will process RSI into a score with the following criteria:
        
           30-70 will be a straight 0
           The buy and sell ranges will be -30 to +30

In [17]:
def rsicheck(c):
    if c > (50+RSIcutoff): # Check if RSI indicates overbought
        return -(c-50-RSIcutoff)   #returns a negative value range -30 to 0 indicating sell ()  
    elif c < (50-RSIcutoff): # Check if RSI indicates oversold
        return -(c-50+RSIcutoff) #returns a positive value range 0 to 30 indicating buy 
    else:
        return 0 # Do not use RSI as an indicator within 50 +- cutoff
    
rsisignal = rsi

for stock in rsisignal:
    rsisignal[stock] = rsisignal[stock].apply(lambda c: rsicheck(c))

### MACD Ranking

MACD Ranking Overview:

    Explanation: 
    1. The MACD is taken by subtracting the long EMA from short EMA
    2. The MACD Signal (named MACDsignal) is an EMA of the MACD

    Buy --> When MACD > MACD Signal
    Sell --> When MACD is < MACD Signal

    Normalize: Percent Difference between MACD and MACD Signal

In [18]:
macdscore = (macd-macdsignal)/macdsignal*100

### MFI Ranking

MFI Ranking Overview:

    Buy --> When MFI <20
    Sell --> When MFI is >80

    Normalize: MFI should already be normalized

In [19]:
def mficheck(c):
    if c > (50+MFIcutoff): # Check if MFI indicates overbought
        return -(c-50-MFIcutoff)   #returns a negative value range -20 to 0 indicating sell ()  
    elif c < (50-MFIcutoff): # Check if MFI indicates oversold
        return -(c-50+MFIcutoff) #returns a positive value range 0 to 20 indicating buy 
    else:
        return 0 # Do not use MFI as an indicator within 50 +- cutoff
    
mfisignal = mfi

for stock in mfisignal:
    mfisignal[stock] = mfisignal[stock].apply(lambda c: mficheck(c))

# Rank Stocks for Today 

In [20]:
def StockRank(date):
    
    #STEP 1: Filter half of  by equally weighting MFI and RSI (e.g. Identify Oversold Stock)
    score1 = mfisignal+rsisignal
    step1 = score1.loc[date].sort_values(ascending = False)
    step1 = step1.iloc[:UNIVERSE_SIZE//2]
    # print(score1.loc[date])
    # print(step1)
    filteredList = step1.index
    # print(filteredList)
   
    
    #STEP 2: Rank remaining stocks based on equal weightage of EMA, SMA, and MACD
    score2 = emasignal + smasignal + macdscore
    step2 = score2[filteredList].loc[date].sort_values(ascending=False)
    return(step2[:basket_size])

In [21]:
target = StockRank(datetime.datetime.now().date()-datetime.timedelta(1))

In [1]:
print(target)

NameError: name 'target' is not defined

# Establish Connection 

In [23]:
from ib_insync import *

In [24]:
util.startLoop()

In [25]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

#For paper trading account, must use Delayed/Frozen Data which is selected with integer 4
ib.reqMarketDataType(4)

# Helper Functions 

### Buy (Limit) 

In [26]:
#Important Note: Soemtimes trade executes aboce Ask price
#Probably due to delayed market data

def purchase_stock(stock, quantity):
    contract = Stock(stock, 'SMART', 'USD')
    ib.qualifyContracts(contract)
    
    ticker = ib.reqMktData(contract, genericTickList = '221, 225, 233')
    
    #sleep for 0.01 seconds to allow ticker to update
    ib.sleep(0.01)
    
    price = ticker.bid
    order = LimitOrder('BUY', quantity, price)
    trade = ib.placeOrder(contract, order)
    
    ib.sleep(3)
    while trade.isDone() == False:
        ib.cancelOrder(order)
        price += 0.1
        order = LimitOrder('BUY', quantity, price)
        trade = ib.placeOrder(contract, order)
        ib.sleep(3)

### Sell (Limit)

In [27]:
#Important Note: Soemtimes trade executes aboce Ask price
#Probably due to delayed market data

def sell_stock(stock, quantity):
    contract = Stock(stock, 'SMART', 'USD')
    ib.qualifyContracts(contract)
    
    ticker = ib.reqMktData(contract, genericTickList = '221, 225, 233')
    
    #sleep for 0.01 seconds to allow ticker to update
    ib.sleep(0.01)
    
    price = ticker.ask
    order = LimitOrder('SELL', quantity, price)
    trade = ib.placeOrder(contract, order)
    
    ib.sleep(3)
    while trade.isDone() == False:
        ib.cancelOrder(order)
        price -= 0.1
        order = LimitOrder('SELL', quantity, price)
        trade = ib.placeOrder(contract, order)
        ib.sleep(3)

### Get Portfolio Positions 

In [28]:
def update_portfolio():
    global portfolio
    portfolio = pd.DataFrame()
    portfolio.insert(0, 'Index', ['Quantity','AvgCost','MktPrice', 'MktValue'])
    portfolio.set_index('Index',inplace = True)
    portfolio[:] = 0  
    ib_portfolio = ib.portfolio()
    for item in ib_portfolio:
        stock_name = item.contract.symbol
        stock_quantity = item.position
        stock_averageCost = item.averageCost
        stock_marketPrice = item.marketPrice
        stock_marketValue = item.marketValue
        portfolio[stock_name] = [stock_quantity, stock_averageCost,
                                 stock_marketPrice, stock_marketValue]

### Liquidate

In [30]:
def liquidate(target):
        update_portfolio()
        overlap_stock = []
        
        for stock in portfolio:
            #If you own the stock and it is not going to be repurchased --> fully liquidate position
            if portfolio[stock]['Quantity'] > 0 and stock not in target.index:
                sell_stock(stock, portfolio[stock]['Quantity'])
            
            #Figure out which shares are 'overlapping'
            elif stock in target.index:
                overlap_stock.append(stock)
       
        #accountSummary[19] returns net liquidation value of portfolio
        net_value = float(ib.accountSummary()[53].value)
        part = net_value/basket_size
                
        for stock in overlap_stock:
            #If your position is greater than its allocated percentage (part), sell off the excess
            if (portfolio[stock]['MktValue'] > part):
                balanced_quantity = part // portfolio[stock]['MktPrice']
                sell_quantity = portfolio[stock]['Quantity'] - balanced_quantity
                sell_stock(stock, sell_quantity)
        update_portfolio()
        print(overlap_stock)
    

### Purchase

In [31]:
 def purchaser(target):
        update_portfolio()
        net_value = float(ib.accountSummary()[53].value)
        part = net_value/basket_size
        
        for stock in target.index:
            #Only purchase NEW stock (ignore overlapped stock)
            if stock not in portfolio:
                #Calculate amount using last availible close price
                buy_quantity = part // close[stock].iloc[-1]
                purchase_stock(stock, buy_quantity)
            elif stock in portfolio:
                if (portfolio[stock]['MktValue'] < part):
                    balanced_quantity = part // portfolio[stock]['MktPrice']
                    buy_quantity = balanced_quantity - portfolio[stock]['Quantity'] 
                    purchase_stock(stock, buy_quantity)
                    
        update_portfolio()

# Main Program 

In [32]:
update_portfolio()
print("Current Portfolio:")
print(portfolio)

print("Target Stocks are: ", target)

print("Step 1: Liquidate Old Positions")
liquidate(target)

print("/-----Step 1 Completed-----/")

print("Step 2: Purchase New Positions")
purchaser(target)

print("/-----Step 2 Completed-----/")

update_portfolio()
print(portfolio)

Current Portfolio:
                  ADBE           AIG            BK           CRM  \
Index                                                              
Quantity    191.000000   3416.000000   2686.000000    362.000000   
AvgCost     497.181847     29.336131     36.110874    254.137776   
MktPrice    523.205566     28.895998     36.666378    272.116058   
MktValue  99932.260000  98708.730000  98485.890000  98506.010000   

                  CSCO           CVX           EXC            GM  \
Index                                                              
Quantity   2357.000000   1161.000000   2688.000000   3382.000000   
AvgCost      42.176102     85.240228     36.819572     28.639873   
MktPrice     41.956001     85.093994     36.666000     29.156000   
MktValue  98890.300000  98794.130000  98558.210000  98605.590000   

                   JNJ           XOM  
Index                                 
Quantity    654.000000   2449.000000  
AvgCost     152.029611     40.580133  
MktPric

Error 321, reqId 313: Error validating request:-'bN' : cause - The size value cannot be zero:
Canceled order: Trade(contract=Stock(conId=5684, symbol='CVX', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='CVX', tradingClass='CVX'), order=LimitOrder(orderId=313, clientId=1, action='BUY', lmtPrice=85.17), orderStatus=OrderStatus(orderId=313, status='Cancelled', filled=0, remaining=0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2020, 8, 26, 15, 45, 18, 419304, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2020, 8, 26, 15, 45, 18, 419964, tzinfo=datetime.timezone.utc), status='Cancelled', message="Error 321, reqId 313: Error validating request:-'bN' : cause - The size value cannot be zero:")])
Error 321, reqId 316: Error validating request:-'bN' : cause - The size value cannot be zero:
Canceled order: 

/-----Step 2 Completed-----/
                  ADBE           AIG            BK           CRM  \
Index                                                              
Quantity    188.000000   3420.000000   2695.000000    363.000000   
AvgCost     497.181847     29.335937     36.113146    254.190096   
MktPrice    522.559326     28.906095     36.674000    272.032501   
MktValue  98241.150000  98858.850000  98836.430000  98747.800000   

                  CSCO           CVX           EXC            GM  \
Index                                                              
Quantity   2355.000000   1161.000000   2695.000000   3382.000000   
AvgCost      42.176102     85.240228     36.819555     28.639873   
MktPrice     41.973999     85.143997     36.680000     29.170000   
MktValue  98848.770000  98852.180000  98852.600000  98652.940000   

                   JNJ           XOM  
Index                                 
Quantity    654.000000   2451.000000  
AvgCost     152.029611     40.580260

# Disconnect 

In [33]:
ib.disconnect()

# Old Buy/Sell Functions (Market)

### Buy Function (Market)

In [ ]:
def purchase_stock(stock, quantity):
    contract = Stock(stock, 'SMART', 'USD')
    ib.qualifyContracts(contract)
    order = MarketOrder('BUY', quantity)
    
    trade = ib.placeOrder(contract, order)

### Sell Function (Market)

In [ ]:
def sell_stock(stock, quantity):
    contract = Stock(stock, 'SMART', 'USD')
    ib.qualifyContracts(contract)
    order = MarketOrder('SELL', quantity)
    
    trade = ib.placeOrder(contract, order)